# Unistats – Merging Datasets

One of the advantages of working with the XML representation of the data is that each record may contain *all the data* associated with a particular provider. However, the same data is also split, perhaps less conveniently, across several CSV files. The contents of these files are "normalised" data tables (in the sense of *database normalisation*)  which means that data is not duplicated across different files although common identifiers do exist that allow data from different tables to be combined back together.

*[__Database normalisation__](https://en.wikipedia.org/wiki/Database_normalization) is a technique for representing tabular dataset across several data tables to help improve data integrity. Different levels of normalisation (different "normal forms") are possible and offer different pros and cons when storing and working with data.*

In the Unistats CSV data files, lots of the files contain references to the UKPRN, the UK Provider Reference Number. UKPRNs are unique identifiers that identify different providers, but how do we know *which* provider they refer to?

We're going to work with the data using the *pandas* package, so let's import it in the conventional way:

In [2]:
import pandas as pd

As an example dataset, consider the location dataset that provides the central co-ordinates of university site locations:

In [3]:
locations_df = pd.read_csv("on_2021_08_11_07_24_51/LOCATION.csv")

locations_df.head()

,UKPRN,ACCOMURL,ACCOMURLW,LOCID,LOCNAME,LOCNAMEW,LATITUDE,LONGITUDE,LOCUKPRN,LOCCOUNTRY,SUURL,SUURLW
0,10000055,https://www.brookes.ac.uk/studying-at-brookes/...,NaN,AB,Abingdon &amp; Witney College (Abingdon Campus),NaN,51.680769,-1.286935,10000055.0,XF,https://www.nus.org.uk/en/students-unions/abin...,NaN
1,10000055,https://www.brookes.ac.uk/studying-at-brookes/...,NaN,CL,Abingdon &amp; Witney College (Common Leys Cam...,NaN,51.820200,-1.477227,10000055.0,XF,https://www.nus.org.uk/en/students-unions/abin...,NaN
2,10000055,https://www.brookes.ac.uk/studying-at-brookes/...,NaN,WI,Abingdon &amp; Witney College (Witney Campus),NaN,51.786705,-1.487938,10000055.0,XF,https://www.nus.org.uk/en/students-unions/abin...,NaN
3,10000163,http://www.aecc.ac.uk/undergrad/accommodation....,NaN,AE,AECC University College,NaN,50.727180,-1.828588,NaN,XF,http://www.aecc.ac.uk/undergrad/su/suhome.aspx,NaN
4,10000216,https://www.allnations.ac.uk/fees-sheet,NaN,AN,All Nations Christian College,NaN,51.802520,0.000933,10000216.0,XF,https://www.allnations.ac.uk/community-life/st...,NaN


One thing we notice about this dataset is that it gives us the name of site location, but not the institution.

We also notice that the site name looks a bit tatty - ampersands are HTML encoded as `&amp;`, for example. We can tidy that up by creating a simple lambda function "formula" that uses a string `.replace(OLDSTRING, NEWSTRING)` function to replace the `&amp;` occurrences by the simpler `&`:

In [5]:
# Replace HTML encoded ampersand with a simple ampersand
# Also take the opportunity to .strip() any whitespace around the location name
clean_ampersand = lambda x: x.replace("&amp;", "&").strip()

clean_ampersand("Abingdon &amp; Witney College (Abingdon Campus) ")

'Abingdon & Witney College (Abingdon Campus)'

We can apply this function to the `LOCNAME` column in out locations dataframe, reassigning the cleaned location names back to the original column:

In [6]:
locations_df["LOCNAME"] = locations_df["LOCNAME"].apply(clean_ampersand)

locations_df.head()

,UKPRN,ACCOMURL,ACCOMURLW,LOCID,LOCNAME,LOCNAMEW,LATITUDE,LONGITUDE,LOCUKPRN,LOCCOUNTRY,SUURL,SUURLW
0,10000055,https://www.brookes.ac.uk/studying-at-brookes/...,NaN,AB,Abingdon & Witney College (Abingdon Campus),NaN,51.680769,-1.286935,10000055.0,XF,https://www.nus.org.uk/en/students-unions/abin...,NaN
1,10000055,https://www.brookes.ac.uk/studying-at-brookes/...,NaN,CL,Abingdon & Witney College (Common Leys Campus),NaN,51.820200,-1.477227,10000055.0,XF,https://www.nus.org.uk/en/students-unions/abin...,NaN
2,10000055,https://www.brookes.ac.uk/studying-at-brookes/...,NaN,WI,Abingdon & Witney College (Witney Campus),NaN,51.786705,-1.487938,10000055.0,XF,https://www.nus.org.uk/en/students-unions/abin...,NaN
3,10000163,http://www.aecc.ac.uk/undergrad/accommodation....,NaN,AE,AECC University College,NaN,50.727180,-1.828588,NaN,XF,http://www.aecc.ac.uk/undergrad/su/suhome.aspx,NaN
4,10000216,https://www.allnations.ac.uk/fees-sheet,NaN,AN,All Nations Christian College,NaN,51.802520,0.000933,10000216.0,XF,https://www.allnations.ac.uk/community-life/st...,NaN


*We might also note that the `LOCUKPRN` hsense of at as been cast as a decimal number rather than an integer because of the presence of at least one `NaN` null value in that column. We won't be referring to that column in this notebook, so we don't really need to figure out a way of cleaning it back to an integer value right now, but we note it nonetheless.*

We don't necessarily need all the columns, so let's simplify the dataframe a little and update our reference to the data so that it only contains a subset of the original data:

In [9]:
cols = ["UKPRN", "LOCID", "LOCNAME", "LATITUDE", "LONGITUDE"]

locations_df = locations_df[cols]
locations_df.head()

,UKPRN,LOCID,LOCNAME,LATITUDE,LONGITUDE
0,10000055,AB,Abingdon & Witney College (Abingdon Campus),51.680769,-1.286935
1,10000055,CL,Abingdon & Witney College (Common Leys Campus),51.820200,-1.477227
2,10000055,WI,Abingdon & Witney College (Witney Campus),51.786705,-1.487938
3,10000163,AE,AECC University College,50.727180,-1.828588
4,10000216,AN,All Nations Christian College,51.802520,0.000933


## Annotating the Data With Provider Names

From the previous data download conversation, you might remember that a lookup table of provider names against UKPRNs was provided as an additional dataset. Since we know what sheet the data is in in the original spreadsheet file, we can load that sheet in directly by calling the *pandas* `.read_excel(FILENAME, SHEETNAME)` function with just the filename and the sheetname:

In [10]:
ukprns = pd.read_excel("UNISTATS_UKPRN_lookup_20160901.xlsx", "Lookup")

ukprns.head()

,UKPRN,NAME
0,10000291,Anglia Ruskin University
1,10000385,The Arts University Bournemouth
2,10000571,Bath Spa University
3,10000712,University College Birmingham
4,10000824,Bournemouth University


How then can we enrich, or annotate, the location data with the provider names?

In each dataframe, we note the presence of the common `UKPRN` column.

In a Microsoft Excel spreadsheet, you might combine data from two sheets using a `=VLOOKUP` formula, `INDEX MATCH` construction or the *Merge Tables* wizard.

In *pandas*, we can use the `pd.merge(LEFT_DATAFRAME, RIGHT_DATAFRAME, on=COMMON_COLNAME)` function to merge the values from two dataframes based on common values in a column with the same name in each dataframe.

*We can also specify the columns used from merging from the left and right dataframes using the `left_on=COLNAME1` and `right_on=COLNAME2` parameters.*

In [12]:
annotated_locations_df = pd.merge(locations_df, ukprns, on="UKPRN")

annotated_locations_df

,UKPRN,LOCID,LOCNAME,LATITUDE,LONGITUDE,NAME
0,10000055,AB,Abingdon & Witney College (Abingdon Campus),51.680769,-1.286935,Abingdon and Witney College
1,10000055,CL,Abingdon & Witney College (Common Leys Campus),51.820200,-1.477227,Abingdon and Witney College
2,10000055,WI,Abingdon & Witney College (Witney Campus),51.786705,-1.487938,Abingdon and Witney College
3,10000163,AE,AECC University College,50.727180,-1.828588,AECC Chiropractic College
4,10000248,AN,ALRA North,53.541119,-2.637748,Academy of Live and Recorded Arts
...,...,...,...,...,...,...
811,10040812,HA,Harper Adams University,52.780019,-2.425380,Harper Adams University
812,10042570,MC,"MidTown Campus, 190 High Holborn",51.516764,-0.122973,Pearson College
813,99999997,ME,Medway Campus,51.396930,0.539673,Medway School of Pharmacy
814,99999998,Y1,The University of York,53.948333,-1.053611,Hull York Medical School


Even if the data in the different columns is misaligned, the `pd.merge()` function will ensure that data from each of the two tables are merged on the basis of having the same unique UKPRN.